# Getting started with DoWhy: A simple example
This is quick introduction to DoWhy causal inference library.
We will load in a sample dataset and estimate causal effect from a (pre-specified)treatment variable to a (pre-specified) outcome variable.

First, let us add required path for python to find DoWhy code and load required packages.

In [1]:
import os, sys
sys.path.append(os.path.abspath("../../"))

In [2]:
import numpy as np
import pandas as pd

import dowhy
from dowhy.do_why import CausalModel
import dowhy.datasets 

Let us first load a dataset. For simplicity, we simulate a dataset with linear relationships between common causes and treatment, and common causes and outcome. 

Beta is the true causal effect. 

In [17]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5,
        num_instruments = 2,
        num_samples=10000, 
        treatment_is_binary=True)
df = data["df"]
print(df.head())
print(data["dot_graph"])

    Z0        Z1        X0        X1        X2        X3        X4    v  \
0  0.0  0.619129  2.226492  1.392440 -0.289724 -1.023950  0.743092  1.0   
1  1.0  0.386653  1.151686 -1.725829  1.129189 -1.206542  0.476580  1.0   
2  0.0  0.191009  0.263892  0.284951 -0.707140 -0.210546 -2.181717  0.0   
3  1.0  0.485629 -1.028494 -0.732026 -2.171436 -0.931260  0.205507  0.0   
4  0.0  0.399060 -0.965090  1.671290  0.036712 -0.786400  0.135370  1.0   

           y  
0  13.508124  
1   9.905194  
2  -4.491918  
3 -18.170923  
4   8.134593  
digraph { v ->y; U[label="Unobserved Confounders"]; U->v; U->y;X0-> v; X1-> v; X2-> v; X3-> v; X4-> v;X0-> y; X1-> y; X2-> y; X3-> y; X4-> y;Z0-> v; Z1-> v;}


Note that we are using a pandas dataframe to load the data. At present, DoWhy only supports pandas dataframe as input.

## Interface 1 (recommended): Input causal graph

We now input a causal graph in the DOT graph format.

In [21]:
# With graph
model=CausalModel(
        data = df,
        treatment=data["treatment_name"],
        outcome=data["outcome_name"],
        graph=data["dot_graph"],
        )

Model to find the causal effect of treatment v on outcome y


In [22]:
model.view_model()

<img src="causal_model.png">

We get the same causal graph. Now identification and estimation is done as before.

In [23]:
identified_estimand = model.identify_effect()
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:{'X2', 'Z0', 'U', 'X1', 'X4', 'X0', 'Z1', 'X3'}


There are unobserved common causes. Causal effect cannot be identified.
WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:['Z1', 'Z0']


Estimand type: ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
d                                      
──(Expectation(y|X2,Z0,X1,X4,X0,Z1,X3))
dv                                     
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,X2,Z0,X1,X4,X0,Z1,X3,U) = P(y|v,X2,Z0,X1,X4,X0,Z1,X3)
### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(y, Z1)/Derivative(v, Z1))
Estimand assumption 1, Exclusion: If we remove {Z1,Z0}→v, then ¬(Z1,Z0→y)
Estimand assumption 2, As-if-random: If U→→y then ¬(U →→Z1,Z0)



In [7]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

LinearRegressionEstimator


INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X2+Z0+X1+X4+X0+Z1+X3


*** Causal Estimate ***

## Target estimand
Estimand type: ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
d                                      
──(Expectation(y|X2,Z0,X1,X4,X0,Z1,X3))
dv                                     
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,X2,Z0,X1,X4,X0,Z1,X3,U) = P(y|v,X2,Z0,X1,X4,X0,Z1,X3)
### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(y, Z1)/Derivative(v, Z1))
Estimand assumption 1, Exclusion: If we remove {Z1,Z0}→v, then ¬(Z1,Z0→y)
Estimand assumption 2, As-if-random: If U→→y then ¬(U →→Z1,Z0)

## Realized estimand
b: y~v+X2+Z0+X1+X4+X0+Z1+X3
## Estimate
Value: 9.999999999999167

Causal Estimate is 10.0


## Interface 2: Specify common causes and instruments

In [8]:
# Without graph                                       
model= CausalModel(                             
        data=df,                                      
        treatment=data["treatment_name"],             
        outcome=data["outcome_name"],                 
        common_causes=data["common_causes_names"])                         

Model to find the causal effect of treatment v on outcome y


In [9]:
model.view_model()

<img src="causal_model.png" />

The above causal graph shows the assumptions encoded in the cauasl model. We can now use this graph to first identify 
the causal effect (go from a causal estimand to a probability expression), and then estimate the causal effect.

**DoWhy philosophy: Keep identification and estimation separate**

Identification can be achieved without access to data, only the graph. This results in an expression to computed. This expression can then be computed using the available data in the estimation step.
Important to understand that these are orthogonal steps.

* Identification

In [10]:
identified_estimand = model.identify_effect()                         

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:{'X2', 'U', 'X1', 'X0', 'X4', 'X3'}


There are unobserved common causes. Causal effect cannot be identified.
WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


* Estimation

In [11]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", 
        test_significance=True
        )         
print(estimate)
print("Causal Estimate is " + str(estimate.value))

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X2+X1+X0+X4+X3


LinearRegressionEstimator
*** Causal Estimate ***

## Target estimand
Estimand type: ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
d                                
──(Expectation(y|X2,X1,X0,X4,X3))
dv                               
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,X2,X1,X0,X4,X3,U) = P(y|v,X2,X1,X0,X4,X3)
### Estimand : 2
Estimand name: iv
No such variable found!

## Realized estimand
b: y~v+X2+X1+X0+X4+X3
## Estimate
Value: 9.999999999999115

## Statistical Significance
p-value: 0.0

Causal Estimate is 10.0


## Refuting the estimate

Now refuting the obtained estimate.

### Adding a random common cause variable

In [12]:
res_random=model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
print(res_random)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X2+X1+X0+X4+X3+w_random


Refute: Add a Random Common Cause
Estimated effect:(9.9999999999991154,)
New effect:(9.99999999999911,)



### Replacing treatment with a random (placebo) variable

In [13]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~placebo+X2+X1+X0+X4+X3


Refute: Use a Placebo Treatment
Estimated effect:(9.9999999999991154,)
New effect:(0.10873156114653899,)



### Removing a random subset of the data

In [14]:
res_subset=model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter", subset_fraction=0.9)
print(res_subset)


INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X2+X1+X0+X4+X3


Refute: Use a subset of data
Estimated effect:(9.9999999999991154,)
New effect:(10.00000000000092,)



As you can see, the linear regression estimator is very sensitive to simple refutations.